In [2]:
import ROOT as r

Welcome to JupyROOT 6.24/06


In [7]:
f=r.TFile("/data/maclwong/Ben_Bkg_Samples/v2/user.bhodkins.700401.Znunugamma.mc20e.v2.0_ANALYSIS.root/user.bhodkins.42165174._000001.ANALYSIS.root","RO")
t=f.Get("analysis")

h_istruth_it_ii_pt=r.TH1F("istruth_istight_isiso_pt","istight_isiso",100,0,100)
h_istruth_it_ni_pt=r.TH1F("istruth_istight_noiso_pt","istight_noiso",100,0,100)
h_istruth_nt_ii_pt=r.TH1F("istruth_notight_isiso_pt","notight_isiso",100,0,100)
h_istruth_nt_ni_pt=r.TH1F("istruth_notight_noiso_pt","notight_noiso",100,0,100)

h_notruth_it_ii_pt=r.TH1F("notruth_istight_isiso_pt","istight_isiso",100,0,100)
h_notruth_it_ni_pt=r.TH1F("notruth_istight_noiso_pt","istight_noiso",100,0,100)
h_notruth_nt_ii_pt=r.TH1F("notruth_notight_isiso_pt","notight_isiso",100,0,100)
h_notruth_nt_ni_pt=r.TH1F("notruth_notight_noiso_pt","notight_noiso",100,0,100)

eventcount=0
totalevents=t.GetEntriesFast()
for e in t:

    eventcount+=1
    if eventcount%10000 == 0:
        print(f"Processed {eventcount:6d}/{totalevents} events")
    
    #=================================================================
    # preselection
    #
    met=e.met_met_NOSYS
    met_phi=e.met_phi_NOSYS
    met_tlv=r.TLorentzVector()
    met_tlv.SetPtEtaPhiM(met,0,met_phi,0)

    # MET requirement, for now just look at low MET
    if e.met_met_NOSYS>250000: continue

    # veto events with leptons
    nelectrons = len(e.el_pt_NOSYS)
    nmuons     = len(e.mu_pt_NOSYS)
    if (nelectrons+nmuons)>0: continue

    # no b-tagged jets, at least one jet with pT>100 GeV, no jets near MET
    jets=[]
    mindphijetsmet=999
    btags=0
    for j in range(len(e.jet_pt_NOSYS)):
        tlv=r.TLorentzVector()
        tlv.SetPtEtaPhiM(e.jet_pt_NOSYS[j],
                         e.jet_eta[j],
                         e.jet_phi[j],
                         0)
        jets.append(tlv)
        mindphijetsmet=min(mindphijetsmet,tlv.DeltaPhi(met_tlv))
        if ord(e.jet_btag_select[j])!=0: btags+=1
    if jets[0].Pt()<100000: continue
    if mindphijetsmet<0.4: continue
    if btags>0: continue

    # at least one photon with pT>10 GeV, no photons near jets
    photons=[]
    mindrphjet=999
    for p in range(len(e.ph_pt_NOSYS)):
        tlv=r.TLorentzVector()
        tlv.SetPtEtaPhiM(e.ph_pt_NOSYS[p],
                         e.ph_eta[p],
                         e.ph_phi[p],
                         0)
        photons.append(tlv)     
        for j in jets:
            mindrphjet=min(mindrphjet,tlv.DeltaPhi(j))
    if mindrphjet<0.4: continue
    if len(photons)<1 or photons[0].Pt()<10000: continue

    # overlap removal for vgamma samples
    if not e.in_vgamma_overlap_7: continue
    #=================================================================

    #=================================================================
    # now find the photons that we want to use
    #
    photon_index=-1
    for p in range(len(e.ph_pt_NOSYS)):
        tlv=r.TLorentzVector()
        tlv.SetPtEtaPhiM(e.ph_pt_NOSYS[p],
                         e.ph_eta[p],
                         e.ph_phi[p],
                         0)
        if e.ph_pt_NOSYS[p]<10000:
            continue
        if e.ph_isEM_NOSYS[p] & 0x45fc01 != 0:
            continue
        photon_index=p
        break
    #=================================================================

    #=================================================================
    # now we have the index of the photon to keep.  Now figure out if this is a truth-matched photon or not.
    # will need to fix this whenever Giordon et al decide on a truth matching scheme.
    ph_truthmatch=(e.ph_truthType[photon_index]!=16 and e.ph_truthpdgId[photon_index]==22)

    ph_tight=e.ph_select_tightID_NOSYS[photon_index]
    ph_iso=e.ph_select_tightIso_NOSYS[photon_index]

    if ph_truthmatch:
        if       ph_tight and     ph_iso: h_istruth_it_ii_pt.Fill(e.ph_pt_NOSYS[photon_index]/1000.)
        elif     ph_tight and not ph_iso: h_istruth_it_ni_pt.Fill(e.ph_pt_NOSYS[photon_index]/1000.)
        elif not ph_tight and     ph_iso: h_istruth_nt_ii_pt.Fill(e.ph_pt_NOSYS[photon_index]/1000.)
        elif not ph_tight and not ph_iso: h_istruth_nt_ni_pt.Fill(e.ph_pt_NOSYS[photon_index]/1000.)
    else:
        if       ph_tight and     ph_iso: h_notruth_it_ii_pt.Fill(e.ph_pt_NOSYS[photon_index]/1000.)
        elif     ph_tight and not ph_iso: h_notruth_it_ni_pt.Fill(e.ph_pt_NOSYS[photon_index]/1000.)
        elif not ph_tight and     ph_iso: h_notruth_nt_ii_pt.Fill(e.ph_pt_NOSYS[photon_index]/1000.)
        elif not ph_tight and not ph_iso: h_notruth_nt_ni_pt.Fill(e.ph_pt_NOSYS[photon_index]/1000.)
    #=================================================================
    
print(f" truth,  tight,  isolated: {h_istruth_it_ii_pt.GetSumOfWeights()}")
print(f" truth,  tight, !isolated: {h_istruth_it_ni_pt.GetSumOfWeights()}")
print(f" truth, !tight,  isolated: {h_istruth_nt_ii_pt.GetSumOfWeights()}")
print(f" truth, !tight,  isolated: {h_istruth_nt_ni_pt.GetSumOfWeights()}")

print(f"!truth,  tight,  isolated: {h_notruth_it_ii_pt.GetSumOfWeights()}")
print(f"!truth,  tight, !isolated: {h_notruth_it_ni_pt.GetSumOfWeights()}")
print(f"!truth, !tight,  isolated: {h_notruth_nt_ii_pt.GetSumOfWeights()}")
print(f"!truth, !tight,  isolated: {h_notruth_nt_ni_pt.GetSumOfWeights()}")

Processed  10000/872532 events
Processed  20000/872532 events
Processed  30000/872532 events
Processed  40000/872532 events
Processed  50000/872532 events
Processed  60000/872532 events
Processed  70000/872532 events
Processed  80000/872532 events
Processed  90000/872532 events
Processed 100000/872532 events
Processed 110000/872532 events
Processed 120000/872532 events
Processed 130000/872532 events
Processed 140000/872532 events
Processed 150000/872532 events
Processed 160000/872532 events
Processed 170000/872532 events
Processed 180000/872532 events
Processed 190000/872532 events
Processed 200000/872532 events
Processed 210000/872532 events
Processed 220000/872532 events
Processed 230000/872532 events
Processed 240000/872532 events
Processed 250000/872532 events
Processed 260000/872532 events
Processed 270000/872532 events
Processed 280000/872532 events
Processed 290000/872532 events
Processed 300000/872532 events
Processed 310000/872532 events
Processed 320000/872532 events
Processe

Error in <THashList::Delete>: A list is accessing an object (0x8293450) already deleted (list name = THashList)
